In [ ]:
from utils_behavior import Sleap_utils
from utils_behavior import HoloviewsTemplates
from utils_behavior import HoloviewsPlots
from utils_behavior import Utils
from utils_behavior import Processing

from utils_behavior import Ballpushing_utils


import importlib

from pathlib import Path

import json

from matplotlib import pyplot as plt
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import seaborn as sns
from shiny import App, Inputs, Outputs, Session, reactive, render, req, ui



#import lux

import pandas as pd

#lux.config.set_executor_type("Pandas")

import numpy as np
import h5py
import re

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
Datapath = Path("/mnt/upramdya_data/MD/F1_Tracks/Videos")

# Look for video folders and load Fly objects

# videos = list(Datapath.rglob("*.mp4"))

# Flies = []

# for video in videos:
#     fly = Ballpushing_utils.Fly(video.parent)
    
#     Flies.append(fly)

# For each directory in the datapath, load the Experiment object
Experiments = []

for exp in Datapath.iterdir():
    if exp.is_dir():
        experiment = Ballpushing_utils.Experiment(exp, experiment_type="F1")
        Experiments.append(experiment)

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
Data_F1 = Ballpushing_utils.Dataset(Experiments, dataset_type="F1_coordinates" )

str(Data_F1)

In [ ]:
Data_F1_coordinates = Data_F1.data

In [ ]:
# Save dataset as feather

Data_F1_coordinates.to_feather("/mnt/upramdya_data/MD/F1_Tracks/Datasets/F1_coordinates.feather")

In [ ]:
Data_F1_coordinates["fly"].nunique()

In [ ]:
Data_F1_coordinates.head()

In [ ]:
# Keep only data where adjusted time is >= 0

Data_F1_coordinates = Data_F1_coordinates[Data_F1_coordinates["adjusted_time"] >= 0]

Data_F1_coordinates = Data_F1_coordinates[Data_F1_coordinates["adjusted_time"] <= 2700]

Permutation Test

In [ ]:
Data_F1_coordinates['F1_condition'].unique()

In [ ]:
Subset = Data_F1_coordinates[Data_F1_coordinates["F1_condition"]!= "pretrained_unlocked"]

In [ ]:
bins = 10

Subset_binned = Processing.preprocess_data(Subset, time_col = "adjusted_time", value_col="test_ball", group_col="F1_condition", bins = bins) 

In [ ]:
permutation_test = Processing.compute_permutation_test(Subset_binned, metric = "avg_test_ball",group_col="F1_condition", control_group="control")

permutation_test

In [ ]:
Data_F1_coordinates.head()

In [ ]:
# Smooth high frequency noise in the data

Data_F1_coordinates["test_ball_smooth"] = Data_F1_coordinates["test_ball"].rolling(window=10, min_periods=1).mean()

In [ ]:
# Save the dataset to a csv file

Data_F1_coordinates.to_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241128_F1_Dataset.csv")

In [ ]:
Data_Avg = Data_F1_coordinates.groupby(["F1_condition", "time"]).mean("test_ball").reset_index()

# Data_Avg["test_ball_smooth"] = Processing.savgol_lowpass_filter(
#     Data_Avg["test_ball"], window_length=221, polyorder=1
# )

Data_Avg


In [ ]:
# Identify the time at which adjusted time is 0 for each fly
zero_adjusted_time = Data_F1_coordinates.groupby('fly').apply(lambda df: df.loc[df['adjusted_time'].abs().idxmin(), 'time']).reset_index()
zero_adjusted_time.columns = ['fly', 'zero_time']

# Merge the zero_adjusted_time with the original DataFrame
Data_F1_coordinates = Data_F1_coordinates.merge(zero_adjusted_time, on='fly')

# Shift the time values
Data_F1_coordinates['shifted_time'] = Data_F1_coordinates['time'] - Data_F1_coordinates['zero_time']

In [ ]:
# make a rounded adjusted time column where the time is rounded to the nearest 10 seconds

#Data_F1_coordinates["rounded_adjusted_time"] = Data_F1_coordinates["adjusted_time"].round(-1)

# Round adjusted time values to the nearest 10 seconds
Data_F1_coordinates['rounded_adjusted_time'] = (Data_F1_coordinates['adjusted_time'] / 10).round() * 10

In [ ]:
Lineplot = sns.lineplot(data=Data_F1_coordinates, x="rounded_adjusted_time", y="test_ball", hue="F1_condition")

plt.savefig(f"{plot_savepath}/241128_F1_Lineplot.png")

plt.show()

In [ ]:
# Plot the average test ball distance grouped by condition over adjusted time with adjusted time starting at 0

sns.lineplot(data=Data_Avg, x="adjusted_time", y="test_ball_smooth", hue="F1_condition", ci="sd")

# Add the 95% confidence interval

#plt.fill_between(Data_Avg["adjusted_time"], Data_Avg["smoothed_test_ball_upper"], Data_Avg["smoothed_test_ball_lower"], alpha=0.3)

plt.show()

# Exploring the metrics

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
Data_F1_metrics = Ballpushing_utils.Dataset(Experiments, dataset_type="summary" )

str(Data_F1_metrics)

In [ ]:
Data_metrics  = Data_F1_metrics.data

In [ ]:
Data_metrics.head()

In [ ]:
# Save the dataset to a csv file

Data_metrics.to_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241128_F1_Summary.csv")

In [ ]:
# Load the dataset from the csv file

Data_metrics = pd.read_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241115_F1_Summary.csv")

In [ ]:
Data_metrics

In [ ]:
Data_metrics.columns

In [ ]:
# Plot the pulling ratio for training as x and for test as y, colored by F1 condition

# Data_Training = Data_metrics[(Data_metrics["F1_condition"] != "control") & (Data_metrics["ball_condition"] == "training")]
# Data_Test = Data_metrics[(Data_metrics["F1_condition"] != "control") & (Data_metrics["ball_condition"] == "test")]

Data_Training = Data_metrics[(Data_metrics["ball_condition"] == "training")]
Data_Test = Data_metrics[(Data_metrics["ball_condition"] == "test")]

merged = Data_Training.merge(Data_Test, on="fly")

In [ ]:
merged

In [ ]:
plot_savepath = "/mnt/upramdya_data/MD/F1_Tracks/Plots/250203_NewPlots"

In [ ]:
# Seaborn styling

sns.set_theme(style="whitegrid")

In [ ]:
plt.figure(figsize=(20, 20))
# Create the jointplot
g = sns.jointplot(data=merged, x='pulling_ratio_x', y='pulling_ratio_y', hue='F1_condition_x')

# Set the title and labels using the JointGrid object
g.fig.suptitle('Pulling Ratio: Training vs Test', y=1.03)
g.set_axis_labels('Pulling Ratio (Training)', 'Pulling Ratio (Test)')

# Save the plot before showing it
plt.savefig(f"{plot_savepath}/Pulling_Ratio_Training_vs_Test.png", bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Save the plot
plt.savefig(f"{plot_savepath}/Pulling_Ratio_Training_vs_Test.png")


In [ ]:
Data_Test = Data_metrics[(Data_metrics["ball_condition"] == "test")]

In [ ]:
# Save the dataset to a csv file

Data_Test.to_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241128_F1_Summary_Test.csv")

In [ ]:
# Let's get rid of the flies that never exit the corridor, so for which exit_time is None

Data_metrics = Data_metrics[Data_metrics["exit_time"].notnull()]

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

# Create the figure and axes
plt.figure(figsize=(10, 6))

# Create the boxplot
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='final_event_time', hue='F1_condition')

# Overlay the strip plot
sns.stripplot(data=Data_Test, x='F1_condition', y='final_event_time', color="black", dodge=False, jitter=True, ax=bp)

# Set the title
plt.title('Final Event Time: Test Ball')

# Save the plot with bbox_inches='tight' to avoid cropping
#plt.savefig(f"{plot_savepath}/Final_Event_Time_Test_Ball.png", bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

# Create the figure and axes
plt.figure(figsize=(10, 6))

# Create the boxplot
bp = sns.boxplot(data=Data_Test, y='F1_condition', x='max_event_time', color='black', fill=False)

# Overlay the strip plot
sns.stripplot(data=Data_Test, y='F1_condition', x='max_event_time', hue="F1_condition", dodge=False, jitter=True, ax=bp, alpha=0.75)

# Set the title
plt.title('Final Event Time: Test Ball')

# Save the plot with bbox_inches='tight' to avoid cropping
plt.savefig(f"{plot_savepath}/max_event_time_Test_Ball.png", bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
importlib.reload(HoloviewsTemplates)

In [ ]:
# HV version

hv_bp = HoloviewsTemplates.jitter_boxplot(Data_Test, sort_by="F1_condition", metric= 'final_event_time', kdims= 'F1_condition')




hv_bp

In [ ]:
# Here let's compute bootstrap confidence intervals for the mean of the max event time grouped by F1 condition

intervals = {}

for condition in Data_Test["F1_condition"].unique():
    condition_data = Data_Test[Data_Test["F1_condition"] == condition]
    
    BS = Processing.draw_bs_ci(data=condition_data["final_event_time"].dropna(), func=np.mean, n_reps=1000)
    
    intervals[condition] = BS
    

# Compute effect size between pretrained and control BS

effect_size = Processing.compute_effect_size(intervals["control"], intervals["pretrained"])

effect_size


In [ ]:
# Plot the confidence intervals

fig = plt.figure(figsize=(10, 6))

for condition in intervals.keys():
    
    plt.errorbar(x=condition, y=np.mean(intervals[condition]), yerr=np.std(intervals[condition]), fmt='o', label=condition)
    
plt.title('Final event time: Test Ball')

#plt.savefig(f"{plot_savepath}/Final_Event_Time_Test_Ball_CI.png", bbox_inches='tight')

plt.show()

In [ ]:
# Let's now transform these value in h:m:s format

import datetime

average_time = effect_size[0]

print(f"Average time difference between control and pretrained is: {datetime.timedelta(seconds=average_time)}")

interval_times = effect_size[1]

print(f"bootstrapped 95% confidence interval for the difference is: {datetime.timedelta(seconds=interval_times[1])} - {datetime.timedelta(seconds=interval_times[0])}")

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='max_event_time', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='max_event_time', color = "black", dodge=False, jitter=True)

plt.title('Max event time: Test Ball')

plt.savefig(f"{plot_savepath}/Max_Event_Time_Test_Ball.png", bbox_inches='tight')

plt.show()

In [ ]:
# Here let's compute bootstrap confidence intervals for the mean of the max event time grouped by F1 condition

intervals = {}

for condition in Data_Test["F1_condition"].unique():
    condition_data = Data_Test[Data_Test["F1_condition"] == condition]
    
    BS = Processing.draw_bs_ci(data=condition_data["max_event_time"].dropna(), func=np.mean, n_reps=1000)
    
    intervals[condition] = BS
    
    


In [ ]:
intervals

In [ ]:
# Plot the confidence intervals

fig = plt.figure(figsize=(10, 6))

for condition in intervals.keys():
    
    plt.errorbar(x=condition, y=np.mean(intervals[condition]), yerr=np.std(intervals[condition]), fmt='o', label=condition)
    
plt.title('Max event time: Test Ball')

plt.savefig(f"{plot_savepath}/Max_Event_Time_Test_Ball_CI.png", bbox_inches='tight')

plt.show()

In [ ]:
importlib.reload(Processing)

In [ ]:
# Compute effect size between pretrained and control BS

effect_size = Processing.compute_effect_size(intervals["control"], intervals["pretrained"])

effect_size

In [ ]:
# Let's now transform these value in h:m:s format

import datetime

average_time = effect_size[0]

print(f"Average time difference between control and pretrained is: {datetime.timedelta(seconds=average_time)}")

interval_times = effect_size[1]

print(f"bootstrapped 95% confidence interval for the difference is: {datetime.timedelta(seconds=interval_times[1])} - {datetime.timedelta(seconds=interval_times[0])}")



In [ ]:
Data_Test.columns

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='aha_moment', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='aha_moment', color = "black", dodge=False, jitter=True)

plt.title('Pulling ratio: Aha moment')

#plt.savefig(f"{plot_savepath}/Aha_Test_Ball.png", bbox_inches='tight')

plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='aha_moment_time', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='aha_moment_time', color = "black", dodge=False, jitter=True)

plt.title('Pulling ratio: Aha moment')

#plt.savefig(f"{plot_savepath}/Aha_Test_Ball_time.png", bbox_inches='tight')

plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, y='F1_condition', x='aha_moment_time', color='black', fill=False)
sns.stripplot(data=Data_Test, y='F1_condition', x='aha_moment_time', hue = "F1_condition", dodge=False, jitter=True, alpha=0.75)

plt.title('Pulling ratio: Aha moment')

plt.savefig(f"{plot_savepath}/Aha_Test_Ball_time.png", bbox_inches='tight')

plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='cumulated_breaks_duration', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='cumulated_breaks_duration', color = "black", dodge=False, jitter=True)

#plt.title('Pulling ratio: Aha moment')

#plt.savefig(f"{plot_savepath}/Adjusted_NbEvents.png", bbox_inches='tight')

plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
bp = sns.boxplot(data=Data_Test, x='F1_condition', y='pulling_ratio', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='pulling_ratio', color = "black", dodge=False, jitter=True)

plt.title('Pulling ratio: Test Ball')

plt.savefig(f"{plot_savepath}/Pulling_Ratio_Test_Ball.png", bbox_inches='tight')

plt.show()

In [ ]:
# Let's make histograms of the pulling ratio for each F1 condition

plt.figure(figsize=(10, 6))

for condition in Data_Test["F1_condition"].unique():
    
    condition_data = Data_Test[Data_Test["F1_condition"] == condition]
    
    sns.histplot(data=condition_data, x='pulling_ratio', kde=True, label=condition)
    
plt.title('Pulling ratio: Test Ball')

plt.show()

In [ ]:
# Now let's make them in 3 different subplots

fig, axs = plt.subplots(1, 3, figsize=(20, 6))

for i, condition in enumerate(Data_Test["F1_condition"].unique()):
    
    condition_data = Data_Test[Data_Test["F1_condition"] == condition]
    
    sns.histplot(data=condition_data, x='pulling_ratio', kde=True, label=condition, ax=axs[i])
    
    axs[i].set_title(condition)
    
plt.suptitle('Pulling ratio: Test Ball')

plt.show()

In [ ]:
# Make a jitter boxplot of first significant time as function of F1 condition

plt.figure(figsize=(10, 6))
sns.boxplot(data=Data_Test, x='F1_condition', y='pulled', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='pulled', color = "black", dodge=False, jitter=True)

plt.title('Pulling events: Test Ball')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=Data_Test, x='F1_condition', y='pushed', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='pushed', color = "black", dodge=False, jitter=True)

plt.title('Pushing events: Test Ball')


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=Data_Test, x='F1_condition', y='max_distance', hue='F1_condition')
sns.stripplot(data=Data_Test, x='F1_condition', y='max_distance', color = "black", dodge=False, jitter=True)

plt.title('max_distance: Test Ball')

Counter intuitively, the max distance the ball was moved for pretrained is lower

In [ ]:
# Get bootstrap confidence intervals for the mean of the max distance

intervals = {}

for condition in Data_Test["F1_condition"].unique():
    condition_data = Data_Test[Data_Test["F1_condition"] == condition]
    
    BS = Processing.draw_bs_ci(data=condition_data["max_distance"].dropna(), func=np.mean, n_reps=1000)
    
    intervals[condition] = BS

In [ ]:
# Plot the confidence intervals

plt.figure(figsize=(10, 6))

for condition in intervals.keys():
        
        plt.errorbar(x=condition, y=np.mean(intervals[condition]), yerr=np.std(intervals[condition]), fmt='o', label=condition)
        
plt.title('Max distance: Test Ball')

plt.show()

In [ ]:
# Compute effect size between pretrained and control BS

effect_size = Processing.compute_effect_size(intervals["control"], intervals["pretrained"])

effect_size

In [ ]:
# Get how many flies are in each success_direction category, grouped by F1 condition

Data_Test.groupby(["F1_condition", "success_direction"]).size()

Interestingly, there are very little groups that both push and pull the ball significantly. Seems to be one way or the other. 

There could be atleast 2 reasons for that: 

1) The flies have different strategies and stick to them

2) The way the ball looks when it has already been pushed or pulled once is leading to more of the same behavior (some kind of stigmery)

One way to check this is to verify what are the prefered behaviors for the first ball for pretrained

In [ ]:
# For each fly, make a new column that is "match" if the success_direction is the same according for both ball conditions or "different" if it is different

Data_Test.groupby(["F1_condition", "direction_match"]).size()

When only considering strictly different condiditions (ignoring "both" as being push AND pull), I get: 
F1_condition         direction_match
pretrained           different          85
                     match              13
pretrained_unlocked  different          59
                     match               8


If I consider both as partial match, I get: 
F1_condition         direction_match
pretrained           different          44
                     match              13
                     partial_match      41
pretrained_unlocked  different           6
                     match               8
                     partial_match      53


so pretrained have 13 / 98 matches, if counting partial matches as well we get : 54 / 98 (half)
pretrained_unlocked have 8 / 67 match, if counting the partial matches: 61/67

Conclusion: for pretrained_unlocked they probably have much more "both" in the beginning, so that bumps up the partial matches

In [ ]:
# Let's compare success_direction for training and test ball

Data_Training.groupby(["F1_condition", "success_direction"]).size()

Here we have 34 / 96 push, goes up to 76 / 96 if we include both. There's quite a lot of pull as well

In [ ]:
# For pretrained flies, compare fly exit time according to success_direction

plt.figure(figsize=(10, 6))

sns.boxplot(data=Data_Training, x='F1_condition', y='exit_time', hue='success_direction')

As expected, the 20 pull I see for pretrained are the flies that didn't exit (None time) and flies that didn't show pulling (push condition) exited earlier compared to the ones that pulled atleast once. 

> What if we remove the fail flies from the dataset? What will be the consequence?

In [ ]:
Clean_Test = Data_Test.dropna(subset=["exit_time"])

Clean_Test.groupby(["F1_condition", "success_direction"]).size()

In [ ]:
29/51

Now we have for pretrained a majority of pushers, 21 / 36 : 0.58

for pretrained unlocked, same as before as they all exited: 29 /51 : 0.57

Comparable.

In [ ]:
Clean_Test.groupby(["F1_condition", "direction_match"]).size()

Now we get 11/46, with partial match : 34 / 46 : majority are matching.

pretrained unlocked have almost only partial match, didn't change.

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
Data_F1 = Ballpushing_utils.Dataset(Experiments)

In [ ]:
# Generate the checkpoints dataset

Data_Checkpoints = Data_F1.generate_dataset("F1_checkpoints")

In [ ]:
# Get number of unique flies

Data_Checkpoints["fly"].nunique()

In [ ]:
Data_Checkpoints

In [ ]:
# Order the F1_conditions to be pretrained, control, pretrained_unlocked

Data_Checkpoints["F1_condition"] = pd.Categorical(
    Data_Checkpoints["F1_condition"],
    categories=["pretrained", "control", "pretrained_unlocked"],
    ordered=True,
)

In [ ]:
# Save as csv

Data_Checkpoints.to_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241128_F1_Checkpoints.csv")

In [ ]:
#Data_Checkpoints["adjusted_time"] = Data_Checkpoints["adjusted_time"] - Data_Checkpoints["fly_exit_time"]

In [ ]:
# Set the theme and style
sns.set_theme(rc={'figure.figsize':(20,12)})
sns.set_style("whitegrid")

# Create the boxplot with larger boxes
jitterbox = sns.boxplot(x="distance", y="adjusted_time", hue="F1_condition", data=Data_Checkpoints, gap = 0.1, fliersize=0) 

# Overlay the strip plot with jittered points
jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color="black", data=Data_Checkpoints, dodge=True, jitter=0.1, size=3)

# Adjust the legend to avoid duplication
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[0:len(handles)//2], labels[0:len(labels)//2], title="Condition")

# Set the title
jitterbox.set_title("Distance to Checkpoints over Time")

plt.savefig(f"{plot_savepath}/Distance_to_Checkpoints.png", bbox_inches='tight')

plt.show()

In [ ]:
# Create the boxplot with larger boxes
jitterbox = sns.boxplot(x="distance", y="adjusted_time", hue="F1_condition", data=Data_Checkpoints, fliersize=0)

# Add light gray lines for each individual
for name, group in Data_Checkpoints.groupby("fly"):
    plt.plot(group["distance"], group["adjusted_time"], color="lightgray", linewidth=0.5, zorder=0)


# Overlay the strip plot with jittered points
sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color="black", data=Data_Checkpoints, dodge=True, jitter=0.1, size=3, ax=jitterbox)

# Adjust the legend to avoid duplication
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[0:len(handles)//2], labels[0:len(labels)//2], title="Condition")

# Set the title
plt.title("Distance to Checkpoints over Time")

# Save the plot with bbox_inches='tight' to avoid cropping
#plt.savefig(f"{plot_savepath}/Distance_to_Checkpoints.png", bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
# Now do the same but split it between flies for which Data_Checkpoints["success_direction"] == "push" and Data_Checkpoints["success_direction"] == "pull"

Data_Checkpoints_push = Data_Checkpoints[Data_Checkpoints["success_direction"] == "push"]

Data_Checkpoints_pull = Data_Checkpoints[Data_Checkpoints["success_direction"] == "pull"]

Data_Checkpoints_both = Data_Checkpoints[Data_Checkpoints["success_direction"] == "both"]

# Set the theme and style
sns.set_theme(rc={'figure.figsize':(20,12)})
sns.set_style("whitegrid")

# Create the boxplot with larger boxes

jitterbox = sns.boxplot(x="distance", y="adjusted_time", hue="F1_condition", data=Data_Checkpoints_push, gap = 0.1, fliersize=0)

# Overlay the strip plot with jittered points

jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color="black", data=Data_Checkpoints_push, dodge=True, jitter=0.1, size=3)

# Adjust the legend to avoid duplication

handles, labels = plt.gca().get_legend_handles_labels()

plt.legend(handles[0:len(handles)//2], labels[0:len(labels)//2], title="Condition")

# Set the title

jitterbox.set_title("Distance to Checkpoints over Time: Push")

plt.savefig(f"{plot_savepath}/Distance_to_Checkpoints_Push.png", bbox_inches='tight')

plt.show()

In [ ]:



# Set the theme and style
sns.set_theme(rc={'figure.figsize':(20,12)})
sns.set_style("whitegrid")

# Create the boxplot with larger boxes

jitterbox = sns.boxplot(x="distance", y="adjusted_time", hue="F1_condition", data=Data_Checkpoints_pull, gap = 0.1, fliersize=0)

# Overlay the strip plot with jittered points

jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color="black", data=Data_Checkpoints_pull, dodge=True, jitter=0.1, size=3)

# Adjust the legend to avoid duplication

handles, labels = plt.gca().get_legend_handles_labels()

plt.legend(handles[0:len(handles)//2], labels[0:len(labels)//2], title="Condition")

# Set the title

jitterbox.set_title("Distance to Checkpoints over Time: Push")

plt.savefig(f"{plot_savepath}/Distance_to_Checkpoints_Pull.png", bbox_inches='tight')

plt.show()

In [ ]:
# Now redo the initial plot but color the points by success direction

# Set the theme and style
sns.set_theme(rc={'figure.figsize':(20,12)})
sns.set_style("whitegrid")

# Define color palettes
boxplot_palette = "Set2"
stripplot_palette = {"push": "blue", "pull": "red", "both": "green", "none": "gray"}

# Create the boxplot with larger boxes
jitterbox = sns.boxplot(x="distance", y="adjusted_time", hue="F1_condition", data=Data_Checkpoints, palette=boxplot_palette, fliersize=0,)

# Overlay the strip plot with jittered points
jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color = "red", data=Data_Checkpoints_pull, dodge=True, jitter=0.1, size=3,)

jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color = "blue", data=Data_Checkpoints_push, dodge=True, jitter=0.1, size=3,)

jitterbox = sns.stripplot(x="distance", y="adjusted_time", hue="F1_condition", color = "yellow", data=Data_Checkpoints_both, dodge=True, jitter=0.1, size=3,)

# Adjust the legend to avoid duplication
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[0:len(handles)//2], labels[0:len(labels)//2], title="Success Direction")

# Set the title
jitterbox.set_title("Distance to Checkpoints over Time")

plt.show()

In [ ]:
# Update the 'pretraining' column based on 'F1_condition'
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "pretrained", "pretraining"] = "y"
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "pretrained_unlocked", "pretraining"] = "y"
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "control", "pretraining"] = "n"

# Update the 'unlocked' column based on 'F1_condition'
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "pretrained", "unlocked"] = "n"
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "pretrained_unlocked", "unlocked"] = "y"
Data_Checkpoints.loc[Data_Checkpoints["F1_condition"] == "control", "unlocked"] = "n"

# Display the first few rows of the DataFrame
Data_Checkpoints.head()




In [ ]:
# Save again

Data_Checkpoints.to_csv("/mnt/upramdya_data/MD/F1_Tracks/Datasets/241114_F1_Checkpoints.csv")